In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
base_url = 'https://www.xxxxx.com'
final_result = list()

def scrap_braun(url, level, data):
    url = base_url + url
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    categories = soup.select('a._cluster-link__link')

    if categories:
        for category in categories:
            next_url = category.attrs['href']
            content = category.select('span._cluster-link__text')[0].contents[0]
            data[f'sub{level}'] = content
            scrap_braun(next_url, level+1, data)
            del data[f'sub{level}']
    else:
        products = soup.select('div._product-tile')
        for product in products:
            tmp = dict()
            descriptionssub = []
            tmp['name'] = product.select('h3._product-tile__name')[0].contents[0].strip()
            fullurl = base_url + product.select('a._product-tile__link')[0].attrs['href']
            html1 = requests.get(fullurl).text
            soup2 = BeautifulSoup(html1, 'html.parser')
            fullurl = soup2.select('article._article _article--product')
            
            scrap_de1 = soup2.find_all("div", {"class": "_product-text _product-text--no-headline _product-text--xbesc"})
            for element1 in scrap_de1:
                d1_elements = element1.find_all("p") or element1.find_all("sup")
                for d1 in d1_elements:
                    de1 = d1.text.strip()
                    descriptionssub.append(de1)
            
            scrap_de2 = soup2.find_all("div", {"class": "_product-text _product-text--xadvn"})
            for element2 in scrap_de2:
                de2 = element2.text.strip()
                de2 = de2.replace("Advantages", "Advantages ::")
                ul_elements = element2.find_all("li")
                for ul in ul_elements:
                    ul_text = ul.text.strip()
                    de2 = de2.replace(ul_text, ul_text + ",")
                descriptionssub.append(de2)

            scrap_de3 = soup2.find_all("div", {"class": "_product-text _product-text--xindi"})
            for element3 in scrap_de3:
                de3 = element3.text.strip()
                de3 = de3.replace("Indication", "Indication ::")
                ul2_elements = element3.find_all("li")
                for ul2 in ul2_elements:
                    ul2_text = ul2.text.strip()
                    de3 = de3.replace(ul2_text, ul2_text + ",")
                descriptionssub.append(de3)

            scrap_de4 = soup2.find_all("div", {"class": "_product-text _product-text--no-headline _product-text--xbesc"})
            for element4 in scrap_de4:
                de42 = element4.find_all("li")
                for i in de42:
                    de41 = i.text.strip()
                    de4 = de41.replace(de41, de41 + ",")
                    descriptionssub.append(de4)

            combined_description = " ".join(descriptionssub)
            tmp['description'] = combined_description
            tmp['link'] = base_url + product.select('a._product-tile__link')[0].attrs['href']
            result = {**data, **tmp}

            print(result)
            final_result.append(result)
            time.sleep(1)

scrap_braun('/en/products-and-solutions/product-catalog.html', 1, {})
df1 = pd.DataFrame(final_result, dtype=str)
df1.to_excel('BXXXX-Product.xlsx', index=False)


{'sub1': 'Continence Care & Urology', 'sub2': 'Urine Catheters', 'sub3': 'Intermittent Catheters', 'sub4': 'Actreen® Intermittent catheters', 'name': 'Actreen® Mini Cath', 'description': "Actreen® Mini Cath is a short urinary catheter designed for female self catheterization. It's an easy and\xa0ready to use catheter with a feminine and modern design. Pouches are included in each box for more discretion in which patients can carry their daily quantity of catheters. Advantages ::\nSterile ready-to-use mini catheter: 9 cm long catheter (length not including connector),Available in CH08 - CH16 (2.7 - 5.3 mm),Pre-lubricated catheter, no need to add water,Easy to open using pre-cut,Easy to use,Smooth eyelets at the tip for deep bladder emptying,Universal connector: Fits all types of urine bags,Short connector allows hygienic catheterization and avoids contact with toilets,PVC-free, DEHP-free,For further discretion pouches are provided in each box. Therefore patients can carry their daily ca